In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as partial
%matplotlib inline

In [2]:
# 导入训练数据和测试数据
df_train = pd.read_csv("train_data_public.csv")
df_test = pd.read_csv("test_public.csv")
df_train = df_train.drop(['id'],axis=1)
df_test = df_test.drop(['id'],axis=1)
df_train.head()

,text,BIO_anno,class
0,交行14年用过，半年准备提额，却直接被降到1Ｋ，半年期间只T过一次三千，其它全部真实消费，第...,B-BANK I-BANK O O O O O O O O O O B-COMMENTS_N...,0
1,单标我有了，最近visa双标返现活动好,B-PRODUCT I-PRODUCT O O O O O O B-PRODUCT I-PR...,1
2,建设银行提额很慢的……,B-BANK I-BANK I-BANK I-BANK B-COMMENTS_N I-COM...,0
3,我的怎么显示0.25费率，而且不管分多少期都一样费率，可惜只有69k,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
4,利率不错，可以撸,B-COMMENTS_N I-COMMENTS_N B-COMMENTS_ADJ I-COM...,1


In [3]:
# 加载TokenEmbedding
from paddlenlp.embeddings import TokenEmbedding

# 初始化TokenEmbedding， 预训练embedding未下载时会自动下载并加载数据
token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300")

# 查看token_embedding详情
print(token_embedding)

c:\Users\Administer\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO 2022-08-25 19:06:37,301 download.py:117] unique_endpoints {''}
INFO 2022-08-25 19:06:37,304 download.py:250] Downloading w2v.baidu_encyclopedia.target.word-word.dim300.tar.gz from https://bj.bcebos.com/paddlenlp/models/embeddings/w2v.baidu_encyclopedia.target.word-word.dim300.tar.gz
100%|██████████| 694483/694483 [04:04<00:00, 2841.91it/s]
INFO 2022-08-25 19:10:42,094 download.py:287] Decompressing C:\Users\Administer\.paddlenlp\models\embeddings\w2v.baidu_encyclopedia.target.word-word.dim300.tar.gz...


In [ ]:
import paddle
import paddle.nn as nn
import paddlenlp

# 构建词袋模型
class BoWModel(nn.Layer):
    def __init__(self, embedder):
        super().__init__()
        self.embedder = embedder
        emb_dim = self.embedder.embedding_dim
        self.encoder = paddlenlp.seq2vec.BoWEncoder(emb_dim)
        self.cos_sim_func = nn.CosineSimilarity(axis=-1)

    def forward(self, text):
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)

        # Shape: (batch_size, embedding_dim)
        summed = self.encoder(embedded_text)

        return summed

model = BoWModel(embedder=token_embedding)

Building prefix dict from the default dictionary ...
DEBUG 2022-08-25 18:30:32,080 __init__.py:113] Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
DEBUG 2022-08-25 18:30:32,944 __init__.py:146] Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.020 seconds.
DEBUG 2022-08-25 18:30:33,103 __init__.py:164] Loading model cost 1.020 seconds.
Prefix dict has been built successfully.
DEBUG 2022-08-25 18:30:33,104 __init__.py:166] Prefix dict has been built successfully.


['工商银行',
 '的',
 '信用卡',
 '是',
 '经典',
 '信用卡',
 '中',
 '的',
 '常青树',
 '，',
 '在',
 '工行',
 '区',
 '的',
 '讨论',
 '热度',
 '也',
 '是',
 '常年',
 'TOP',
 ' ',
 '1',
 '。']

In [ ]:
from data import Tokenizer
tokenizer = Tokenizer()
tokenizer.set_vocab(vocab=token_embedding.vocab)